# LangChain 1.0 Middleware

Ref:

- https://docs.langchain.com/oss/python/langchain/middleware

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain.chat_models import init_chat_model
import mlflow
import os

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = init_chat_model(
    "openai:databricks-gpt-oss-20b",
    api_key=creds.token,
    base_url=creds.host + "/serving-endpoints",
)

# 以下のように環境変数設定でもできる
# os.environ["OPENAI_API_KEY"] = creds.token
# os.environ["OPENAI_BASE_URL"] = creds.host + "/serving-endpoints"

# model = init_chat_model("openai:databricks-gpt-oss-20b")
# model.invoke("Hello")


In [0]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""

    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

## Middleware: Summarization

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware


checkpointer = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model=model,
            max_tokens_before_summary=1000,  # Trigger summarization at 4000 tokens
            messages_to_keep=5,  # Keep last 20 messages after summary
            summary_prompt="Custom prompt for summarization...",  # Optional
        ),
    ],
)

In [0]:

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    context=Context(user_id="1"),
    config=config,
)

## Middleware: Human in the loop

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware


checkpointer = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "get_user_location": {
                    "allowed_decisions": ["approve"],
                },
            }
        ),
    ],
)

In [0]:
from langgraph.types import Command

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response.get("__interrupt__"))

agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    context=Context(user_id="1"),
    config=config,
)

## Middleware: Model call limit

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import ModelCallLimitMiddleware


checkpointer = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        ModelCallLimitMiddleware(
            thread_limit=5,
            run_limit=2,
            exit_behavior="end",
        ),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response)

## Middleware: Tool call limit

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import ToolCallLimitMiddleware

checkpointer = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        ToolCallLimitMiddleware(
            thread_limit=5,
            run_limit=1,
            exit_behavior="end",
        ),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response)

## Middleware: Model fallback

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import ModelFallbackMiddleware

checkpointer = InMemorySaver()
llm_config = {
    "api_key": creds.token,
    "base_url": creds.host + "/serving-endpoints",
}

agent = create_agent(
    model=init_chat_model("openai:databricks-qwen3-next-80b-a3b-instruct", **llm_config),
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        ModelFallbackMiddleware(
            init_chat_model("openai:databricks-gemma-3-12b", **llm_config),
            init_chat_model("openai:databricks-llama-4-maverick", **llm_config),
        ),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response)

## Middleware: PII

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import PIIMiddleware

checkpointer = InMemorySaver()
agent = create_agent(
    model=init_chat_model("openai:databricks-qwen3-next-80b-a3b-instruct", **llm_config),
    checkpointer=checkpointer,
    middleware=[
        PIIMiddleware("email", strategy="mask", apply_to_input=True, apply_to_output=True),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "taro_yamada@gmail.comのメールアドレスをそのまま抜き出して返してください"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response)

## Middleware: Planning

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import TodoListMiddleware

checkpointer = InMemorySaver()
agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        TodoListMiddleware(),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "最初にtodoを整理してから東京と大阪の天気を調べて、違いを説明して"}]},
    context=Context(user_id="1"),
    config=config,
)

print(response)

In [0]:
response["todos"]

## Middleware: LLM tool selector

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import LLMToolSelectorMiddleware

checkpointer = InMemorySaver()
agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        LLMToolSelectorMiddleware(
            model=model,
            max_tools=1,
        ),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "東京と大阪の天気を調べて、違いを説明して",
            }
        ]
    },
    context=Context(user_id="1"),
    config=config,
)

print(response)

## Middleware: Context editing

In [0]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import ContextEditingMiddleware, ClearToolUsesEdit

checkpointer = InMemorySaver()
agent = create_agent(
    model=model,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    checkpointer=checkpointer,
    middleware=[
        ContextEditingMiddleware(
            edits=[
                ClearToolUsesEdit(trigger=100, keep=1
                                  ),  # Clear old tool uses
            ],
        ),
    ],
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "東京と大阪の天気を調べて、違いを説明して",
            }
        ]
    },
    context=Context(user_id="1"),
    config=config,
)

print(response)